<a href="https://colab.research.google.com/github/SanghunOh/study_machinglearning/blob/main/codes/sanghunoh/reports/Term_2_CNN_MPIE_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Connect Drive

In [1]:
path_root = '/content/drive/MyDrive/datas/'

# 작업 경로 설정
import os
os.chdir(path_root)
!ls

results_CNN_label_1_202205301442.pickle
results_CNN_label_2_202205301442.pickle
results_CNN_label_4_202205301442.pickle


In [2]:
# config param
file_prefix = 'results_CNN_label_'  # file name
save_file = True  # save graph, csv 

In [3]:
import glob

filename_list = []
for file_name in glob.glob(f"./{file_prefix}*.pickle"):
    filename_list.append(file_name)
filename_list

['./results_CNN_label_1_202205301442.pickle',
 './results_CNN_label_4_202205301442.pickle',
 './results_CNN_label_2_202205301442.pickle']

##Load Trained Model

In [4]:
import pickle
results_list = list()
for filename in filename_list:
  with open(filename, 'rb') as f:
    results_list.append(pickle.load(f))
len(results_list)   # model, history, execution_time, _params

ValueError: ignored

##create DataFrame with concate params from results

###결과값을 순서 맞게 나열하기
순서 : params + history values + model layers + training time + history params


In [ ]:
import numpy as np
def resultToValueslist(model, history, training_time, params):
  model_layer_depth = [len(model.layers)]
  history_values = list(np.array(list(history.history.values()))[:,-1])
  history_params = list(history.params.values())
  _training_time = [training_time]
  params_values = list(params.values())
  _value_list = params_values + history_values + model_layer_depth + _training_time + history_params
  return _value_list
# value_list

###학습결과값을 DataFrame으로 만들기

In [ ]:
# merge column epochs (epochs 컬럼 2개임)
import pandas as pd

def mergeColumnWithepochs(dataframe):
  # dataframe = _dataframe.copy()
  _epochs_temp = dataframe['epochs']
  _epochs_temp = _epochs_temp.iloc[:,0]
  dataframe.drop(labels=['epochs', 'verbose'], axis=1, inplace=True)
  _result_df = pd.concat([dataframe, _epochs_temp],axis=1)
  return _result_df

In [ ]:
def valueslistToDataFrame(_result_per_label_list):
  _results_list = list()
  for _results in _result_per_label_list:
    _model, _history, _training_time, _params = _results[0], _results[1], _results[2], _results[3]
    # print(f'Training Time : {_training_time:.3f} sec')
    _results_list.append(resultToValueslist(_model, _history, _training_time, _params))
  _column_name = list(_params.keys())+list(_history.history.keys())+['model_layer_depth','training_time']+list(_history.params.keys())

  # print(f'valueslistToDataFrame : {len(_results_list)}, column_name: {_column_name}')
  _result_df = pd.DataFrame(_results_list, columns=_column_name)
  _result_df = mergeColumnWithepochs(_result_df)
  return _result_df

###label_class 별로 DataFrame을 list 담기

In [ ]:
import pandas as pd

results_df_list = list()
for result_per_label_list in results_list:
  results_df_list.append(valueslistToDataFrame(result_per_label_list))

In [ ]:
# results_df_list[0]

##Evaluation

###Per Label class

In [ ]:
evaluation_columns = ['loss', 'val_loss', 'accuracy', 'val_accuracy']

In [ ]:
def sortEvaluationColumns(_results_df):
  _evaluation_df = _results_df.sort_values(evaluation_columns, ascending = [True, False, True, False])
  print(len(_evaluation_df.columns), _evaluation_df.columns)
  return _evaluation_df

In [ ]:
evaluation_df_list = list()
for results_df in results_df_list:
  evaluation_df_list.append(sortEvaluationColumns(results_df))
# evaluation_df


###draw evaluation

In [ ]:
import matplotlib.pyplot as plt

def drawEvaluation(_evaluation_df, _save_file=False, _label=None):
  plt.rcParams["figure.figsize"] = (15, 5)

  plt.subplot(121)
  _evaluation_df[['loss', 'val_loss']].T.plot(ax=plt.gca())
  plt.grid(axis='y')
  plt.title('loss rate')
  # plt.xticks(rotation=20)

  plt.subplot(122)
  _evaluation_df[['accuracy', 'val_accuracy']].T.plot(ax=plt.gca())
  plt.grid(axis='y')
  # plt.xticks(rotation=20)
  plt.title('accuracy rate')

  _label_class = _label
  if not _label:
    _label_class = _evaluation_df.loc[0,'label_class']
  
  plt.suptitle(f'label_class : {_label_class}')
  if _save_file:
    _file_name = file_prefix + str(_label_class) + '.png'
    plt.savefig(_file_name)
  plt.show()


In [ ]:
for evaluation_df in evaluation_df_list:
  drawEvaluation(evaluation_df, save_file)


###성능 영향 준 Parameters

In [ ]:
def getParametersByPerformance(_evaluation_df, _save_file=False, _label=None):
  _label_class = _label
  if not _label:
    _label_class = _evaluation_df.loc[0,'label_class']

  columns_mask_str = _evaluation_df.columns.map(lambda x: True if (x not in evaluation_columns) else False )
  # print(columns_mask_str, len(list(columns_mask_str)))
  performance_df = _evaluation_df.loc[:,columns_mask_str].value_counts().reset_index()
  performance_df.drop(labels=0, axis=1, inplace=True)

  _measure_columns = list()
  for column in performance_df.columns:
    if len(performance_df[column].unique()) > 1:
      _measure_columns.append(column)
  # print(_measure_columns)
  result_columns = evaluation_columns + _measure_columns
  _evaluation_df = _evaluation_df[result_columns]

  if _save_file:
    _file_name = file_prefix + str(_label_class) + '.csv'
    _evaluation_df.to_csv(_file_name)
  return _evaluation_df

In [ ]:
performance_params_df_list = list()
for evaluation_df in evaluation_df_list:
  performance_params_df_list.append(getParametersByPerformance(evaluation_df, save_file))
# performance_params_df_list[-1]

In [ ]:
performance_params_df_list[0]

##Concate All Dataset for Performance

In [ ]:
results_df_all = pd.concat(results_df_list).reset_index()
results_df_all.drop(labels='index', axis=1, inplace=True)
results_df_all

In [ ]:
evaluation_df_all = sortEvaluationColumns(results_df_all)
evaluation_df_all

In [ ]:
evaluation_df_all_rank = evaluation_df_all[:10]
drawEvaluation(evaluation_df_all_rank, save_file, _label='all')

In [ ]:
getParametersByPerformance(evaluation_df_all_rank, save_file, _label='all')